# ETL2 - SCRIPT DE PIPELINE DE DADOS

##### Script responsável por buscar os dados coletados do COVID19 que estão no Cloud Storage, transformar eles para atender as necessidades do projeto e, por fim, salvá-los na zona de outro do Cloud Storage em formato PARQUET

#### PRIMEIRA PARTE

In [1]:
# Importa as bibliotecas necessárias
import pandas as pd
import numpy as np
import gzip
import requests
import os
import datetime

In [2]:
# Salva os dados contidos no arquivo covid19.gz em um dataframe chamado DADOS
dados = pd.read_csv("gs://igti-pa/bronze/covid19.gz", compression='gzip', sep=',')

In [4]:
# Verifica as informações basicas do dataframe criado
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3190960 entries, 0 to 3190959
Data columns (total 18 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   city                                           object 
 1   city_ibge_code                                 float64
 2   date                                           object 
 3   epidemiological_week                           int64  
 4   estimated_population                           float64
 5   estimated_population_2019                      float64
 6   is_last                                        bool   
 7   is_repeated                                    bool   
 8   last_available_confirmed                       int64  
 9   last_available_confirmed_per_100k_inhabitants  float64
 10  last_available_date                            object 
 11  last_available_death_rate                      float64
 12  last_available_deaths                     

In [5]:
# Exclui as colunas que não serão utilizadas 
dados.drop(['estimated_population', 'estimated_population_2019', 'is_last', 'is_repeated',
            'last_available_confirmed_per_100k_inhabitants', 'order_for_place',
             'last_available_date', 'last_available_death_rate'], axis=1, inplace=True)

In [6]:
# Filtra os dados apenas até 2020 - após 2020 não há necessidade de análise
dados = dados[dados['date'] <= '2020-12-31']

In [7]:
# Existem algumas coletas de dados que não tiveram identificação de cidade, então ficou salvo no campo como NaN
# Altera todos os valores NaN para 0 (int)
dados['city_ibge_code'].fillna(0, inplace=True)

In [8]:
# Altera o código IBGE primeiro para INT para retirar o (.) e, após isso, altera o campo para STR
dados = dados.astype({"city_ibge_code": int})
dados = dados.astype({"city_ibge_code": str})

In [9]:
# Altera o campo do ano+semana da pandemia para STR
dados = dados.astype({"epidemiological_week": str})

In [10]:
# Renomeia as colunas para portugues para melhor entendimento
dados = dados.rename(columns={'city': 'cidade', 'city_ibge_code': 'codigo_ibge', 'date': 'data', 
                              'epidemiological_week': 'ano_semana_pandemia', 
                              'last_available_confirmed': 'qtd_ultimos_casos_disponiveis',
                              'last_available_deaths': 'qtd_ultimas_mortes_disponiveis', 
                              'state': 'estado', 'new_confirmed': 'novos_casos_confirmados', 
                              'new_deaths': 'novas_mortes_confirmados'})

In [11]:
# Verifica novamente as informações basicas do dataframe após as alterações efetuadas
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1320832 entries, 0 to 3144339
Data columns (total 10 columns):
 #   Column                          Non-Null Count    Dtype 
---  ------                          --------------    ----- 
 0   cidade                          1312890 non-null  object
 1   codigo_ibge                     1320832 non-null  object
 2   data                            1320832 non-null  object
 3   ano_semana_pandemia             1320832 non-null  object
 4   qtd_ultimos_casos_disponiveis   1320832 non-null  int64 
 5   qtd_ultimas_mortes_disponiveis  1320832 non-null  int64 
 6   place_type                      1320832 non-null  object
 7   estado                          1320832 non-null  object
 8   novos_casos_confirmados         1320832 non-null  int64 
 9   novas_mortes_confirmados        1320832 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 110.8+ MB


In [12]:
# Mostra os primeiros três registros do dataframe transformado
dados.head(3)

,cidade,codigo_ibge,data,ano_semana_pandemia,qtd_ultimos_casos_disponiveis,qtd_ultimas_mortes_disponiveis,place_type,estado,novos_casos_confirmados,novas_mortes_confirmados
0,Rio Branco,1200401,2020-03-17,202012,3,0,city,AC,3,0
1,NaN,12,2020-03-17,202012,3,0,state,AC,3,0
2,Rio Branco,1200401,2020-03-18,202012,3,0,city,AC,0,0


In [13]:
dados.to_parquet('gs://igti-pa/prata/dados_covid19.parquet')

#### SEGUNDA PARTE

In [14]:
# Faz a leitura dos dados do COVID19 no bucket em formato PARQUET
dados = pd.read_parquet('gs://igti-pa/prata/dados_covid19.parquet')

In [15]:
# Cria um novo dataframe chamado ESTADOS e salva apenas as informações conforme o campo "place_type"
estados = dados[dados['place_type'] == 'state']
estados = estados.reset_index(drop=True)

In [16]:
# Cria um novo dataframe chamado CIDADES e salva apenas as informações conforme o campo "place_type"
cidades = dados[dados['place_type'] == 'city']
cidades = cidades.reset_index(drop=True)

In [17]:
# Exclui as colunas que não serão utilizadas 
estados.drop(['cidade', 'place_type'] , axis=1, inplace=True)

In [18]:
# Exclui as colunas que não serão utilizadas 
cidades.drop(['place_type'] , axis=1, inplace=True)

In [21]:
# Verifica as informações basicas do dataframe CIDADES após as alterações efetuadas
estados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7942 entries, 0 to 7941
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   codigo_ibge                     7942 non-null   object
 1   data                            7942 non-null   object
 2   ano_semana_pandemia             7942 non-null   object
 3   qtd_ultimos_casos_disponiveis   7942 non-null   int64 
 4   qtd_ultimas_mortes_disponiveis  7942 non-null   int64 
 5   estado                          7942 non-null   object
 6   novos_casos_confirmados         7942 non-null   int64 
 7   novas_mortes_confirmados        7942 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 496.5+ KB


In [22]:
# Verifica as informações basicas do dataframe CIDADES após as alterações efetuadas
cidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312890 entries, 0 to 1312889
Data columns (total 9 columns):
 #   Column                          Non-Null Count    Dtype 
---  ------                          --------------    ----- 
 0   cidade                          1312890 non-null  object
 1   codigo_ibge                     1312890 non-null  object
 2   data                            1312890 non-null  object
 3   ano_semana_pandemia             1312890 non-null  object
 4   qtd_ultimos_casos_disponiveis   1312890 non-null  int64 
 5   qtd_ultimas_mortes_disponiveis  1312890 non-null  int64 
 6   estado                          1312890 non-null  object
 7   novos_casos_confirmados         1312890 non-null  int64 
 8   novas_mortes_confirmados        1312890 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 90.1+ MB


In [19]:
# Mostra os primeiros três registros do dataframe ESTADOS transformado
estados.head(3)

,codigo_ibge,data,ano_semana_pandemia,qtd_ultimos_casos_disponiveis,qtd_ultimas_mortes_disponiveis,estado,novos_casos_confirmados,novas_mortes_confirmados
0,12,2020-03-17,202012,3,0,AC,3,0
1,12,2020-03-18,202012,3,0,AC,0,0
2,12,2020-03-19,202012,4,0,AC,1,0


In [20]:
# Mostra os primeiros três registros do dataframe CIDADES transformado
cidades.head(3)

,cidade,codigo_ibge,data,ano_semana_pandemia,qtd_ultimos_casos_disponiveis,qtd_ultimas_mortes_disponiveis,estado,novos_casos_confirmados,novas_mortes_confirmados
0,Rio Branco,1200401,2020-03-17,202012,3,0,AC,3,0
1,Rio Branco,1200401,2020-03-18,202012,3,0,AC,0,0
2,Rio Branco,1200401,2020-03-19,202012,4,0,AC,1,0


In [23]:
# Salva o novo dataframe ESTADOS no bucket do GCP
estados.to_parquet('gs://igti-pa/ouro/estados_covid19.parquet')

In [24]:
# Salva o novo dataframe CIDADES no bucket do GCP
cidades.to_parquet('gs://igti-pa/ouro/cidades_covid19.parquet')

In [25]:
# Lê as primeiras três linhas do arquivo ESTADOS.parquet criado para verificar a integridade do arquivo
pd.read_parquet('gs://igti-pa/ouro/estados_covid19.parquet')

,codigo_ibge,data,ano_semana_pandemia,qtd_ultimos_casos_disponiveis,qtd_ultimas_mortes_disponiveis,estado,novos_casos_confirmados,novas_mortes_confirmados
0,12,2020-03-17,202012,3,0,AC,3,0
1,12,2020-03-18,202012,3,0,AC,0,0
2,12,2020-03-19,202012,4,0,AC,1,0
3,12,2020-03-20,202012,7,0,AC,3,0
4,12,2020-03-21,202012,11,0,AC,4,0
...,...,...,...,...,...,...,...,...
7937,17,2020-12-27,202053,89410,1226,TO,121,1
7938,17,2020-12-28,202053,89794,1229,TO,384,3
7939,17,2020-12-29,202053,90047,1232,TO,253,3
7940,17,2020-12-30,202053,90358,1234,TO,311,2


In [26]:
# Lê as primeiras três linhas do arquivo CIDADES.parquet criado para verificar a integridade do arquivo
pd.read_parquet('gs://igti-pa/ouro/cidades_covid19.parquet')

,cidade,codigo_ibge,data,ano_semana_pandemia,qtd_ultimos_casos_disponiveis,qtd_ultimas_mortes_disponiveis,estado,novos_casos_confirmados,novas_mortes_confirmados
0,Rio Branco,1200401,2020-03-17,202012,3,0,AC,3,0
1,Rio Branco,1200401,2020-03-18,202012,3,0,AC,0,0
2,Rio Branco,1200401,2020-03-19,202012,4,0,AC,1,0
3,Rio Branco,1200401,2020-03-20,202012,7,0,AC,3,0
4,Rio Branco,1200401,2020-03-21,202012,11,0,AC,4,0
...,...,...,...,...,...,...,...,...,...
1312885,Tocantínia,1721109,2020-12-31,202053,422,5,TO,0,0
1312886,Tupirama,1721257,2020-12-31,202053,115,2,TO,0,0
1312887,Tupiratins,1721307,2020-12-31,202053,33,0,TO,0,0
1312888,Wanderlândia,1722081,2020-12-31,202053,460,6,TO,7,0


### FIM DO SCRIPT ETL2